# Registry-powered Searches

Archives register their data sets so that programs can discover them: http://vao.stsci.edu/keyword-search/

The link above is a GUI, so you can play around with it. 

There is also an applications program interface (API) to this service, so that programs can send queries and retrieve search results automatically. The standard search API is a Table Access Protocol service on a relational data model for registries, which is a very low-level interface. Here we have created several simplified tools for common, useful queries into the registry.


In [ ]:
import requests, io, astropy

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## our stuff
import sys

# Use the NASA_NAVO/astroquery
from navo_utils.registry import Registry

Registry query methods exist in an astroquery.vo.Registry() class with different levels of simplicity and power. For example, if you already know you want to search NED, you can get related service URLs as follows. Note that you may get *more* results than you expect, some of which should be easily differentiated by inspection.

In [ ]:
results = Registry.query(source='ned', service_type='cone', debug=True)
print('Found {} results:'.format(len(results)))
print(results[:]['access_url'])
print(results[1]['ivoid'])
print(results.columns)

The Registry.query() method takes arguments which we can use to further filter the results (passed to internal function _build_adql):  

    service_type : "conesearch", "simpleimageaccess", "simplespectralaccess", "tableaccess". May be shortened to "cone", "image", "spectr", or "table" or "tap", respectively.
    keyword      : any keyword contained in ivoid, title, or description
    waveband     : waveband string. Multiple options may be comma-delimited i.e. "optical, infrared"
    source       : any substring in ivoid
    publisher    : the name of any publishing organization
    order_by     : what field to order it by, but then you have to know the names, currently
                    ("waveband","short_name","ivoid","res_description","access_url","reference_url","publisher", service_type")
    logic_string : any other string you want to add to the ADQL where clause, should start with " and "

The results are returned by Registry.query() in an astropy table using the conversion function _astropy_table_from_votable_response(). 



## Waveband Allowed Terms and values in (A)
* gamma-ray:	 	less than 0.1
* X-ray:	 	0.1-100
* EUV:	 	100-1000
* UV:	 	1,000-3,000
* Optical:	 	3,000-10,000
* Infrared:	 	10,000-1,000,000
* Millimeter:	 	10^6 - 10^8
* Radio:	 	over 10^8






The Registry.query_counts() method takes arguments which we can use to see which keyword values might help us narrow down our search, or possibly give us too MANY results (these are passed to internal function _build_counts_adql):

    field      : keyword field for which to see popular values: "waveband", "publisher", "service_type" currently supported.
    minimum    : A minimum count of occurences for the keyword value to use as a cutoff (optional, defaults to 1)

In [ ]:
results = Registry.query_counts('publisher', 15, debug=True)
print(results)

With a 'publisher' field to work from, we can get a narrowed down query:

In [ ]:
results = Registry.query(source='ned', publisher='Extragalactic Database', service_type='cone', debug=True)
print('Found {} results:'.format(len(results)))
print(results[:]['access_url'])


Note we will need to URL-decode the access_url information in our results, as the registry resource standard expects it be encoded.

In [ ]:
from html import unescape

for result in results:
    print(unescape(result['access_url']))

# Appendix: Documentation on the Standards

### Table Access Protocol 
* IVOA standard for RESTful web service access to tabular data
* http://www.ivoa.net/documents/TAP/


### Registry Relational Schema
* IVOA standard for modeling registry metadata for querying with TAP
* http://www.ivoa.net/documents/RegTAP/


### Astronomical Query Data Language (2.0)
* IVOA standard for querying astronomical data in tabular format, with geometric search support
* http://www.ivoa.net/documents/latest/ADQL.html

